In [1]:
import numpy as np
import pandas as pd

In [2]:
#import csv file with columns and turn into DataFrame
mathands = pd.read_csv("mathands_compiled_m8_columns.csv")
mathands_df = pd.DataFrame(mathands)
mathands_df

,Begin Time - ss.msec,End Time - ss.msec,Duration - ss.msec,*TIME,LHGrasp,RHInteract,RHGrasp,Interact Comment,LHInteract,Grasp Comment,LHinteract,File,File Path
0,119.006,119.382,0.376,START,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_mathands_KC.eaf,NaN
1,119.006,138.795,19.789,NaN,NaN,NaN,mg,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_mathands_KC.eaf,NaN
2,119.006,138.796,19.790,NaN,NaN,m,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_mathands_KC.eaf,NaN
3,137.203,137.597,0.394,NaN,mg,NaN,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_mathands_KC.eaf,NaN
4,137.204,137.596,0.392,NaN,NaN,NaN,NaN,NaN,m,NaN,NaN,001_08_071807_07172018EP_mathands_KC.eaf,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,390.126,390.392,0.266,NaN,NaN,m,NaN,NaN,NaN,NaN,NaN,048_08_020409MA_SYNC_CLEAN_mathands_DH.eaf,NaN
2531,394.308,399.769,5.461,NaN,NaN,m,NaN,NaN,NaN,NaN,NaN,048_08_020409MA_SYNC_CLEAN_mathands_DH.eaf,NaN
2532,394.407,399.781,5.374,NaN,NaN,NaN,mg,NaN,NaN,NaN,NaN,048_08_020409MA_SYNC_CLEAN_mathands_DH.eaf,NaN
2533,404.946,405.081,0.135,End Toy Play - by BK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,048_08_020409MA_SYNC_CLEAN_mathands_DH.eaf,NaN


In [3]:
#keep time, interact and file name columns
mathands_df = mathands_df[['Begin Time - ss.msec','End Time - ss.msec','RHInteract','LHInteract', 'LHinteract','File']]

#rename columns 
mathands_df = mathands_df.rename(columns={'Begin Time - ss.msec':'Begin'})
mathands_df = mathands_df.rename(columns={'End Time - ss.msec':'End'})


#rename the file names into just ID and month (i.e.'001_08')
for i in mathands_df['File']:
    newfilename = i[0:6]
    mathands_df['File'] = mathands_df['File'].replace(i,newfilename)

In [4]:
#turn all contents in Interact columns into string
mathands_df[['RHInteract','LHInteract','LHinteract']] = mathands_df[['RHInteract','LHInteract','LHinteract']].astype(str)

#keep only 'm',' a', 'i' in RHInteract and LHInteract
for i in mathands_df['RHInteract']:
    if i == 'nan':
        no_interact = ''
        mathands_df['RHInteract'] = mathands_df['RHInteract'].replace(i,no_interact)  

for i in mathands_df['LHInteract']:
    if i == 'nan':
        no_interact = ''
        mathands_df['LHInteract'] = mathands_df['LHInteract'].replace(i,no_interact)    

#there are two left hand interact columns
#find values that are in LHinteract and move them to LHInteract and remove column 
for index,row in mathands_df.iterrows():
    if row['LHinteract'] != 'nan':
        mathands_df.at[index,'LHInteract']= row['LHinteract']
    else:
        continue
mathands_df = mathands_df.drop(['LHinteract'], axis=1)
        

In [5]:
#create new columns for three toys
#iterate through every row for each toy type both RHInteract and LHInteract

#musical toy
mathands_df['m'] = ""
for index,row in mathands_df.iterrows():
    if row['RHInteract'] == 'm':
        mathands_df.at[index,'m']= 'R'
        
for index,row in mathands_df.iterrows():
    if row['LHInteract'] == 'm':
        mathands_df.at[index,'m']= 'L'

#inanimate toy 
mathands_df['i'] = ""
for index,row in mathands_df.iterrows():
    if row['RHInteract'] == 'i':
        mathands_df.at[index,'i']= 'R'
        
for index,row in mathands_df.iterrows():
    if row['LHInteract'] == 'i':
        mathands_df.at[index,'i']= 'L'

#animate toy  
mathands_df['a'] = ""
for index,row in mathands_df.iterrows():
    if row['RHInteract'] == 'a':
        mathands_df.at[index,'a']= 'R'

for index,row in mathands_df.iterrows():
    if row['LHInteract'] == 'a':
        mathands_df.at[index,'a']= 'L'

In [6]:
#only keep time, toy types, and file columns
#reindex to make sure order is corrrect 
mathands_df = mathands_df.drop(labels=['RHInteract','LHInteract'], axis=1)
columns_order = ['Begin', 'End','m','i','a','File']
mathands_df = mathands_df.reindex(columns=columns_order)

In [7]:
#drop rows with no values
for index,row in mathands_df.iterrows():
    if row['m']=='':
        if row['i']=='':
            if row['a']=='':
                mathands_df = mathands_df.drop(index)
                                

In [8]:
#reset index from 0 without jumps
mathands_df = mathands_df.reset_index()
mathands_df = mathands_df.drop(labels=['index'], axis=1)
mathands_df = mathands_df.reset_index()


In [9]:
#If Being and End times overlap, change value to Both
# create an empty dataframe to store the new rows
new_data = pd.DataFrame(columns=mathands_df.columns)

groups = mathands_df.groupby('File')

# loop through each row in the original data
for i, row in mathands_df.iterrows():
    # check if there is any overlap with the previous row
    if i > 0 and row['File'] == mathands_df.loc[i-1, 'File'] and row['Begin'] < mathands_df.loc[i-1, 'End']:
        # if there is overlap, add a new row to the new_data dataframe with "both" as the value for the corresponding columns
        overlap_start = max(row['Begin'], mathands_df.loc[i-1, 'Begin'])
        overlap_end = min(row['End'], mathands_df.loc[i-1, 'End'])
        new_row = row.copy()
        new_row['Begin'] = overlap_start
        new_row['End'] = overlap_end
        new_row[new_row == 'R'] = 'Both'
        new_row[new_row == 'L'] = 'Both'
        new_data = new_data.append(new_row, ignore_index=True)
        
    # add the original row to the new_data dataframe
    new_data = new_data.append(row, ignore_index=True)


In [10]:
#delete any duplicated rows with the same Begin and End time 
grouped = new_data.groupby('File')

# define a function to apply to each group
def drop_duplicates(group):
    # check for rows with the same begin and end time
    duplicates = group.duplicated(subset=['Begin', 'End'])
    # drop the duplicates
    group = group[~duplicates].drop_duplicates()
    return group

# apply the function to each group
df_cleaned = grouped.apply(drop_duplicates)
mathands_df = df_cleaned.droplevel('File')

In [11]:
mathands_df[0:20]

,index,Begin,End,m,i,a,File
0,0,119.006,138.796,R,,,001_08
1,1,137.204,137.596,Both,,,001_08
3,2,147.201,148.199,,L,,001_08
4,3,147.204,148.199,,Both,,001_08
5,3,147.204,148.594,,R,,001_08
6,4,151.001,152.494,R,,,001_08
7,5,153.901,165.594,,R,,001_08
8,6,156.503,161.496,,Both,,001_08
10,7,161.901,164.498,,,L,001_08
11,8,165.205,166.799,,L,,001_08


In [12]:
#import csv file with columns and turn into DataFrame
babyhands = pd.read_csv("babyhands_compiled_m8_columns.csv")
babyhands_df = pd.DataFrame(babyhands)

In [13]:
babyhands_df

,Begin Time - ss.msec,End Time - ss.msec,*TIME,Motion,LHGrasp,RHInteract,Mouthing,RHGrasp,Interact Comment,LHInteract,Mom Action,Grasp Comment,Mouth,Mom Actions,LHinteract,Mom Grab,File,File Path
0,119.000,142.899,NaN,NaN,NaN,i,NaN,ig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_babyhands_TD.eaf,NaN
1,119.006,119.382,START,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_babyhands_TD.eaf,NaN
2,123.240,139.679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,i,NaN,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_babyhands_TD.eaf,NaN
3,123.557,139.679,NaN,NaN,ig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_babyhands_TD.eaf,NaN
4,124.417,130.089,NaN,NaN,NaN,NaN,im,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_babyhands_TD.eaf,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3370,394.459,394.717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,momgrab,NaN,NaN,048_08_020409MA_SYNC_CLEAN_babyhands_TD.eaf,NaN
3371,400.240,406.797,NaN,NaN,NaN,m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,048_08_020409MA_SYNC_CLEAN_babyhands_TD.eaf,NaN
3372,401.167,403.871,NaN,shak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,048_08_020409MA_SYNC_CLEAN_babyhands_TD.eaf,NaN
3373,404.946,405.081,End Toy Play - by BK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,048_08_020409MA_SYNC_CLEAN_babyhands_TD.eaf,NaN


In [14]:
#keep only time, interact, mom action, and file columns
babyhands_df = babyhands_df[['Begin Time - ss.msec','End Time - ss.msec','RHInteract','LHInteract', 'LHinteract',
                             'Mom Action','Mom Actions', "Mom Grab",'File']]

#rename columns
babyhands_df = babyhands_df.rename(columns={'Begin Time - ss.msec':'Begin'})
babyhands_df = babyhands_df.rename(columns={'End Time - ss.msec':'End'})

#rename the file names into just ID and month (i.e.'001_08')
for i in babyhands_df['File']:
    newfilename = i[0:6]
    babyhands_df['File'] = babyhands_df['File'].replace(i,newfilename)

In [15]:
#turn all contents in both Interact column and Mom columns into string
babyhands_df[['RHInteract', 'LHInteract','LHinteract','Mom Action','Mom Actions','Mom Grab']] = babyhands_df[['RHInteract', 'LHInteract','LHinteract','Mom Action','Mom Actions','Mom Grab']].astype(str) 

#keep only 'm',' a', 'i' in Interact columns 
for i in babyhands_df['RHInteract']:
    if i == 'nan':
        no_interact = ''
        babyhands_df['RHInteract'] = babyhands_df['RHInteract'].replace(i,no_interact)  

for i in babyhands_df['LHInteract']:
    if i == 'nan':
        no_interact = ''
        babyhands_df['LHInteract'] = babyhands_df['LHInteract'].replace(i,no_interact)  
        
#there are two left hand interact colums
#move all non empty values in LHinteract into LHInteract and remove LHinteract
for index,row in babyhands_df.iterrows():
    if row['LHinteract'] != 'nan':
        babyhands_df.at[index,'LHInteract']= row['LHinteract']
    else:
        continue
babyhands_df = babyhands_df.drop(['LHinteract'], axis=1)
        
#there are three Mom columns
#move all non empty values in Mom Action and Mom grab into Mom Actions and only keep Mom Actions
for index,row in babyhands_df.iterrows():
    if row['Mom Action'] != 'nan':
        babyhands_df.at[index,'Mom Actions']= row['Mom Action']
    else:
        continue
babyhands_df = babyhands_df.drop(['Mom Action'], axis=1)

for index,row in babyhands_df.iterrows():        
    if row['Mom Grab'] != 'nan':
        babyhands_df.at[index,'Mom Actions']= row['Mom Grab']
    else:
        continue
babyhands_df = babyhands_df.drop(['Mom Grab'], axis=1)

for i in babyhands_df['Mom Actions']:
    if i == 'nan':
        no_interact = ''
        babyhands_df['Mom Actions'] = babyhands_df['Mom Actions'].replace(i,no_interact)  
        

In [16]:
#create new columns for three toys
#iterate through every row for each toy type both RHInteract and LHInteract

#musical toy
babyhands_df['m'] = ""
for index,row in babyhands_df.iterrows():
    if row['RHInteract'] == 'm':
        babyhands_df.at[index,'m']= 'R'
        
for index,row in babyhands_df.iterrows():
    if row['LHInteract'] == 'm':
        babyhands_df.at[index,'m']= 'L'

#inanimate toy 
babyhands_df['i'] = ""
for index,row in babyhands_df.iterrows():
    if row['RHInteract'] == 'i':
        babyhands_df.at[index,'i']= 'R'
        
for index,row in babyhands_df.iterrows():
    if row['LHInteract'] == 'i':
        babyhands_df.at[index,'i']= 'L'

#animate toy  
babyhands_df['a'] = ""
for index,row in babyhands_df.iterrows():
    if row['RHInteract'] == 'a':
        babyhands_df.at[index,'a']= 'R'

for index,row in babyhands_df.iterrows():
    if row['LHInteract'] == 'a':
        babyhands_df.at[index,'a']= 'L'

In [17]:
#only keep time, toy types, and file columns
#reindex to make sure order is corrrect 
babyhands_df = babyhands_df.drop(labels=['RHInteract','LHInteract'], axis=1)
columns_order = ['Begin', 'End','m','i','a','Mom Actions','File']
babyhands_df = babyhands_df.reindex(columns=columns_order)

In [18]:
#drop rows with no values
for index,row in babyhands_df.iterrows():
    if row['m']=='':
        if row['i']=='':
            if row['a']=='':
                if row['Mom Actions']=='':
                    babyhands_df = babyhands_df.drop(index)
                
#reset index from 0 without jumps
babyhands_df = babyhands_df.reset_index()
babyhands_df = babyhands_df.drop(labels=['index'], axis=1)
babyhands_df = babyhands_df.reset_index()
         


In [19]:
#If Being and End times overlap, change value to Both
# create an empty dataframe to store the new rows
new_data = pd.DataFrame(columns=babyhands_df.columns)

groups = babyhands_df.groupby('File')

# loop through each row in the original data
for i, row in babyhands_df.iterrows():
    # check if there is any overlap with the previous row
    if i > 0 and row['File'] == babyhands_df.loc[i-1, 'File'] and row['Begin'] < babyhands_df.loc[i-1, 'End']:
        # if there is overlap, add a new row to the new_data dataframe with "both" as the value for the corresponding columns
        overlap_start = max(row['Begin'], babyhands_df.loc[i-1, 'Begin'])
        overlap_end = min(row['End'], babyhands_df.loc[i-1, 'End'])
        new_row = row.copy()
        new_row['Begin'] = overlap_start
        new_row['End'] = overlap_end
        new_row[new_row == 'R'] = 'Both'
        new_row[new_row == 'L'] = 'Both'
        new_data = new_data.append(new_row, ignore_index=True)
        
    # add the original row to the new_data dataframe
    new_data = new_data.append(row, ignore_index=True)




In [20]:
new_data

,index,Begin,End,m,i,a,Mom Actions,File
0,0,119.000,142.899,,R,,,001_08
1,1,123.240,139.679,,Both,,,001_08
2,1,123.240,139.679,,L,,,001_08
3,2,139.613,139.679,,,,momgrab,001_08
4,2,139.613,140.573,,,,momgrab,001_08
...,...,...,...,...,...,...,...,...
2235,1551,389.063,394.694,R,,,,048_08
2236,1552,391.466,391.782,Both,,,,048_08
2237,1552,391.466,391.782,L,,,,048_08
2238,1553,394.459,394.717,,,,momgrab,048_08


In [21]:
#delete any duplicated rows with the same Begin and End time 
grouped = new_data.groupby('File')

# define a function to apply to each group
def drop_duplicates(group):
    # check for rows with the same begin and end time
    duplicates = group.duplicated(subset=['Begin', 'End'])
    # drop the duplicates
    group = group[~duplicates].drop_duplicates()
    return group

# apply the function to each group
df_cleaned = grouped.apply(drop_duplicates)
babyhands_df = df_cleaned.droplevel('File')

In [22]:
babyhands_df

,index,Begin,End,m,i,a,Mom Actions,File
0,0,119.000,142.899,,R,,,001_08
1,1,123.240,139.679,,Both,,,001_08
3,2,139.613,139.679,,,,momgrab,001_08
4,2,139.613,140.573,,,,momgrab,001_08
5,3,140.046,140.573,Both,,,,001_08
...,...,...,...,...,...,...,...,...
2234,1551,389.063,389.627,Both,,,,048_08
2235,1551,389.063,394.694,R,,,,048_08
2236,1552,391.466,391.782,Both,,,,048_08
2238,1553,394.459,394.717,,,,momgrab,048_08


In [23]:
#merge babyhands and mathands
matbaby_df = pd.merge(babyhands_df, mathands_df, how='outer', on=['Begin','End'],
                     suffixes = ('_inf','_mom'))


In [24]:
#change all contents in string 
matbaby_df[['m_inf', 'i_inf','a_inf','Mom Actions','File_inf',
                'm_mom', 'i_mom', 'a_mom','File_mom']] = matbaby_df[['m_inf', 'i_inf','a_inf','Mom Actions','File_inf',
                'm_mom', 'i_mom', 'a_mom','File_mom']].astype(str)

In [25]:
#correspond file name of mom and inf to each other
for index,row in matbaby_df.iterrows():
    if row['File_mom'] == 'nan':
        matbaby_df.at[index,'File_mom']= row['File_inf']
    else:
        continue
        
for index,row in matbaby_df.iterrows():   
    if row['File_inf'] == 'nan':
        matbaby_df.at[index,'File_inf']= row['File_mom']
    else:
        continue

#check file names are the same
#if they are, keep only one and rename to just File
for index,row in matbaby_df.iterrows():   
    if row['File_inf'] != row['File_mom']:
        print('file name not the same')
    else:
        continue   
        
matbaby_df = matbaby_df.drop('File_inf', axis=1)
matbaby_df = matbaby_df.rename(columns={'File_mom':'File'})
        

In [26]:
matbaby_df

,index_inf,Begin,End,m_inf,i_inf,a_inf,Mom Actions,index_mom,m_mom,i_mom,a_mom,File
0,0,119.000,142.899,,R,,,NaN,nan,nan,nan,001_08
1,1,123.240,139.679,,Both,,,NaN,nan,nan,nan,001_08
2,2,139.613,139.679,,,,momgrab,NaN,nan,nan,nan,001_08
3,2,139.613,140.573,,,,momgrab,NaN,nan,nan,nan,001_08
4,3,140.046,140.573,Both,,,,NaN,nan,nan,nan,001_08
...,...,...,...,...,...,...,...,...,...,...,...,...
3675,NaN,381.205,381.973,nan,nan,nan,nan,1508,R,,,048_08
3676,NaN,382.610,386.782,nan,nan,nan,nan,1509,L,,,048_08
3677,NaN,389.627,389.795,nan,nan,nan,nan,1510,L,,,048_08
3678,NaN,390.126,390.392,nan,nan,nan,nan,1511,R,,,048_08


In [27]:
#turn all nan values into empty strings
for index,row in matbaby_df.iterrows():
    if row['m_inf'] == 'nan':
        matbaby_df.at[index,'m_inf']= ''
    else:
        continue

for index,row in matbaby_df.iterrows():
    if row['i_inf'] == 'nan':
        matbaby_df.at[index,'i_inf']= ''
    else:
        continue

for index,row in matbaby_df.iterrows():
    if row['a_inf'] == 'nan':
        matbaby_df.at[index,'a_inf']= ''
    else:
        continue
        
for index,row in matbaby_df.iterrows():
    if row['m_mom'] == 'nan':
        matbaby_df.at[index,'m_mom']= ''
    else:
        continue

for index,row in matbaby_df.iterrows():
    if row['i_mom'] == 'nan':
        matbaby_df.at[index,'i_mom']= ''
    else:
        continue

for index,row in matbaby_df.iterrows():
    if row['a_mom'] == 'nan':
        matbaby_df.at[index,'a_mom']= ''
    else:
        continue

for index,row in matbaby_df.iterrows():
    if row['Mom Actions'] == 'nan':
        matbaby_df.at[index,'Mom Actions']= ''
    else:
        continue

In [28]:
#delete rows where interact is empty
for index,row in matbaby_df.iterrows():
    if row['m_inf']=='':
        if row['i_inf']=='':
            if row['a_inf']=='':
                if row['Mom Actions']=='':
                    if row['m_mom']=='':
                        if row['i_mom']=='':
                            if row['a_mom']=='':
                                matbaby_df = matbaby_df.drop(index)
                                
                                
                                
#reset index from 0 without jumps
matbaby_df = matbaby_df.reset_index()
matbaby_df = matbaby_df.drop(labels=['index'], axis=1)

In [29]:
matbaby_df

,index_inf,Begin,End,m_inf,i_inf,a_inf,Mom Actions,index_mom,m_mom,i_mom,a_mom,File
0,0,119.000,142.899,,R,,,NaN,,,,001_08
1,1,123.240,139.679,,Both,,,NaN,,,,001_08
2,2,139.613,139.679,,,,momgrab,NaN,,,,001_08
3,2,139.613,140.573,,,,momgrab,NaN,,,,001_08
4,3,140.046,140.573,Both,,,,NaN,,,,001_08
...,...,...,...,...,...,...,...,...,...,...,...,...
3675,NaN,381.205,381.973,,,,,1508,R,,,048_08
3676,NaN,382.610,386.782,,,,,1509,L,,,048_08
3677,NaN,389.627,389.795,,,,,1510,L,,,048_08
3678,NaN,390.126,390.392,,,,,1511,R,,,048_08


In [30]:
matbaby_df = matbaby_df.drop(labels=['index_mom'], axis=1)
matbaby_df = matbaby_df.drop(labels=['index_inf'], axis=1)

In [33]:
matbaby_df

,Begin,End,m_inf,i_inf,a_inf,Mom Actions,m_mom,i_mom,a_mom,File
0,119.000,142.899,,R,,,,,,001_08
1,123.240,139.679,,Both,,,,,,001_08
2,139.613,139.679,,,,momgrab,,,,001_08
3,139.613,140.573,,,,momgrab,,,,001_08
4,140.046,140.573,Both,,,,,,,001_08
...,...,...,...,...,...,...,...,...,...,...
3675,381.205,381.973,,,,,R,,,048_08
3676,382.610,386.782,,,,,L,,,048_08
3677,389.627,389.795,,,,,L,,,048_08
3678,390.126,390.392,,,,,R,,,048_08


In [34]:
matbaby_df.to_csv(r'matbab.csv', index=False)
